# Restore individual models

In [3]:
%matplotlib inline

import numpy as np
import tensorflow as tf 
import matplotlib.pyplot as plt
import os 
import matplotlib.pyplot as plt
from matplotlib.image import imread
from scipy.misc import imsave
from scipy.misc import imresize
import numpy as np
from numpy import genfromtxt
import tensorflow as tf
from PIL import Image
import pickle
# from tensorflow_vgg import vgg16
import collections
import time
import math

## Load Auto_Encoder_weights

In [13]:
loaded_graph = tf.Graph()
save_model_path = './checkpoints_view_invariant/Au=1_lr=0.0013_nl=2_bs=80.ckpt'
with tf.Session(graph=loaded_graph) as sess:
    # Load model
    loader = tf.train.import_meta_graph(save_model_path + '.meta')
    loader.restore(sess, save_model_path)

    # Get Tensors from loaded model
    Au_1_weight_1 = loaded_graph.get_tensor_by_name('conv_layers/conv_weights_1:0')
    Au_1_bias_1 = loaded_graph.get_tensor_by_name('conv_layers/bias_1:0')
    Au_1_weight_2 = loaded_graph.get_tensor_by_name('conv_layers/conv_weights_2:0')
    Au_1_bias_2 = loaded_graph.get_tensor_by_name('conv_layers/bias_2:0')

In [10]:
loaded_graph = tf.Graph()
save_model_path = './checkpoints_view_invariant/Au=2_lr=0.001_nl=2_bs=80.ckpt'
with tf.Session(graph=loaded_graph) as sess:
    # Load model
    loader = tf.train.import_meta_graph(save_model_path + '.meta')
    loader.restore(sess, save_model_path)

    # Get Tensors from loaded model
    Au_2_weight_2 = loaded_graph.get_tensor_by_name('conv_layers/conv_weights_2:0')
    Au_2_bias_2 = loaded_graph.get_tensor_by_name('conv_layers/bias_2:0')

In [17]:
loaded_graph = tf.Graph()
save_model_path = './checkpoints_view_invariant/Au=3_lr=0.001_nl=2_bs=80_nl=2.ckpt'
with tf.Session(graph=loaded_graph) as sess:
    # Load model
    loader = tf.train.import_meta_graph(save_model_path + '.meta')
    loader.restore(sess, save_model_path)

    # Get Tensors from loaded model
    Au_3_weight_2 = loaded_graph.get_tensor_by_name('conv_layers/conv_weights_2:0')
    Au_3_bias_2 = loaded_graph.get_tensor_by_name('conv_layers/bias_2:0')
    Au_3_deconvweight_2 = loaded_graph.get_tensor_by_name('deconv_layers/deconv_weights_2:0')
    Au_3_debias_2 = loaded_graph.get_tensor_by_name('deconv_layers/debias_2:0')
    Au_3_outweight = loaded_graph.get_tensor_by_name('output_layer/output_weights:0')
    Au_3_outbias = loaded_graph.get_tensor_by_name('output_layer/outbias:0')

KeyError: "The name 'conv_layers/bias_2:0' refers to a Tensor which does not exist. The operation, 'conv_layers/bias_2', does not exist in the graph."

In [21]:
gallery_full = np.load(open(r'../gait_data/OULP_GEI/GEI64x64_full_Seq00', mode='rb'))
probe_full = np.load(open(r'../gait_data/OULP_GEI/GEI64x64_full_Seq01', mode='rb'))
gallery_full = gallery_full/float(255)
probe_full = probe_full/float(255)

In [22]:
# Train X
train_x = np.concatenate([gallery_full[:2800, 0], probe_full[:2800, 0]], axis=0)
train_x = np.reshape(train_x, [train_x.shape[0], train_x.shape[1], train_x.shape[2], 1])
# Train Y
train_y = np.concatenate([gallery_full[:2800, 3], probe_full[:2800, 3]], axis=0)
train_y = np.reshape(train_x, [train_x.shape[0], train_x.shape[1], train_x.shape[2], 1])
print('train_x', train_x.shape, 'train_y', train_y.shape)

# Val X
val_x = gallery_full[2800:, 0]
val_x = np.reshape(val_x, [val_x.shape[0], val_x.shape[1], val_x.shape[2], 1])
# Val Y
val_y = gallery_full[2800:, 3]
val_y = np.reshape(val_y, [val_y.shape[0], val_y.shape[1], val_y.shape[2], 1])
print('val_x', val_x.shape, 'val_y', val_y.shape)

# Test X
test_x = probe_full[2800:, 0]
test_x = np.reshape(test_x, [test_x.shape[0], test_x.shape[1], test_x.shape[2], 1])
# Test Y
test_y = probe_full[2800:, 3]
test_y = np.reshape(test_x, [test_x.shape[0], test_x.shape[1], test_x.shape[2], 1])
print('test_x', test_x.shape, 'test_y', test_y.shape)

train_x (5600, 64, 64, 1) train_y (5600, 64, 64, 1)
val_x (346, 64, 64, 1) val_y (346, 64, 64, 1)
test_x (346, 64, 64, 1) test_y (346, 64, 64, 1)


## Build End to End

In [15]:
weight = tf.Variable(tf.truncated_normal([2,2,2], stddev=0.02))
weight

In [16]:
def conv_layer(inputs_, weight, bias, conv_name, pool_name, bn_name, relu_name, add_bacth_norm=True, is_training=True,
               out_dim=1, conv_kernsize=(2,2), conv_strides=(1,1), pool_size=(2,2), pool_strides=(2,2), keep_prob=0.5):
    ### Encoder
    # e.g. out_dim: 8; kern_size: (3,3); pool_size: (2,2); strides: (2,2);

    # set strides of conv2d
    stride = [1, conv_strides[0], conv_strides[1], 1]
    
    # conv2d
    # filter:[filter_height, filter_width, in_channels, out_channels]
    # input:[batch, in_height, in_width, in_channels]
    conv2d = tf.nn.bias_add(tf.nn.conv2d(inputs_, weight, stride, padding='SAME', name=conv_name), bias)
    
    # Now 28x28x
    if add_bacth_norm:
        conv2d = tf.layers.batch_normalization(conv2d, training=is_training, name=bn_name)
        
    # add activation function
    conv2d = tf.nn.relu(conv2d, name=relu_name)
    
    # add Max pooling
    conv2d_maxpool = tf.nn.max_pool(conv2d, [1,pool_size[0],pool_size[1],1],\
                                    [1,pool_strides[0],pool_strides[1],1], padding='SAME', name=pool_name)
                    
    conv2d_maxpool = tf.layers.dropout(conv2d_maxpool, rate=keep_prob)

    return conv2d_maxpool

def deconv_layer(encoded, up_name, weight, bias, de_conv_name, bn_name, relu_name, add_batch_norm=True, is_training=True,
                 up_size=[(2,2),(4,4),(5,5),(6,6)], out_dim=1, conv_kernsize=(2,2), conv_strides=(1,1), keep_prob=0.5):
    ### Decoder
    upsample = tf.image.resize_nearest_neighbor(encoded, up_size, name=up_name) # up_size: e.g.(7,7)
    
    # set strides of conv2d
    stride = [1, conv_strides[0], conv_strides[1], 1]
    
    # conv2d
    conv2d = tf.nn.bias_add(tf.nn.conv2d(upsample, weight, stride, padding='SAME', name=de_conv_name),\
                   bias)
    
    if add_batch_norm:
        conv2d = tf.layers.batch_normalization(conv2d, training=is_training, name=bn_name)
    
    conv2d = tf.nn.relu(conv2d, name=relu_name)
    
    conv2d = tf.layers.dropout(conv2d, rate=keep_prob)
    
    return conv2d

def output_layer(conv2d, weight, bias, logits_name, bn_name, decoded_name, add_batch_norm=True, is_training=True, 
                 out_dim=1, conv_kernsize=(3,3), conv_strides=(1,1)):
    
    # set strides of conv2d
    stride = [1, conv_strides[0], conv_strides[1], 1]
    
    # conv2d
    logits = tf.nn.bias_add(tf.nn.conv2d(conv2d, weight, stride, padding='SAME', name=logits_name),\
                   bias)
    
    if add_batch_norm:
        conv2d = tf.layers.batch_normalization(conv2d, training=is_training, name=bn_name)
    #Now 28x28x1
    decoded = tf.nn.sigmoid(logits, name=decoded_name)
    
    return logits, decoded

In [18]:
conv_kern_size = (4,4)
conv_strides = (1,1)
pool_size = (2,2)
pool_strides = (2,2)
up_size = [(64,64)]
de_kern_size = (4,4)
de_conv_strides = (1,1)
out_kern_size = (4,4)
out_strides = (1,1)

tf.reset_default_graph()

learning_rate = tf.placeholder(tf.float32, name='learning_rate')
keep_p = tf.placeholder(tf.float32, name="keep_prob")
training = tf.placeholder(tf.bool, name='training')
inputs = tf.placeholder(tf.float32, [None, 64, 64, 1], name='inputs')
targets = tf.placeholder(tf.float32, [None, 64, 64, 1], name='targets')

# 64x64 -- 32x32
with tf.name_scope('conv_layers'):
    maxpool_1 = conv_layer(inputs, Au_1_weight_1, Au_1_bias_1, 'conv_1', 'pool_1', 'bn_1', 'relu_1', add_bacth_norm=False, 
                           is_training=training, out_dim=16, conv_kernsize=kern_size, conv_strides=conv_strides, 
                           pool_size=pool_size, pool_strides=pool_strides, keep_prob=keep_p)
    
    # Now 32x32x16 Hidden Layer 1 Au_1
    conv_hidden_1 = tf.nn.bias_add(tf.nn.conv2d(maxpool_1, Au_1_weight_2, stride, padding='SAME', name='hidden_1'), Au_1_bias_2)
    
    # Now 32x32x8 Hidden Layer 2 Au_2
    conv_hidden_2 = tf.nn.bias_add(tf.nn.conv2d(conv_hidden_1, Au_2_weight_2, stride, padding='SAME', name='hidden_1'), Au_2_bias_2)

    # Now 32x32x8 Hidden Layer 3 Au_3
    conv_hidden_3 = tf.nn.bias_add(tf.nn.conv2d(conv_hidden_2, Au_3_weight_2, stride, padding='SAME', name='hidden_1'), Au_3_bias_2)
    
    # Now 32x32x8 Deconv Layer
    deconv_1 = deconv_layer(conv_hidden_3, up_name, Au_3_deconvweight_2, Au_3_debias_2, 'deconv_1', 'debn_1', 'derelu_1', add_batch_norm=True, is_training=training,
                 up_size=up_size[0], out_dim=16, conv_kernsize=de_kern_size, conv_strides=de_conv_strides, keep_prob=keep_p)

    # Now 64x64x16 Output Layer
    logits, decoded = output_layer(conv2d, Au_3_outweight, Au_3_outbias, 'logits', 'outbn', 'output', add_batch_norm=True, is_training=training, 
                                   out_dim=1, conv_kernsize=out_kern_size, conv_strides=out_strides)
    
    # Now 64x64x1 Output
    loss = tf.nn.sigmoid_cross_entropy_with_logits(labels=targets, logits=logits, name='loss')
    cost = tf.reduce_mean(loss, name='cost')
    opt = tf.train.AdamOptimizer(learning_rate).minimize(cost)
    
#     with tf.name_scope('saver'):
#         saver = tf.train.Saver()

NameError: name 'Au_1_bias_1' is not defined

In [19]:
def get_batches(data, target, batch_size):
    n_batches = len(data)//batch_size
    data = data[:batch_size * n_batches]
    target = target[:batch_size * n_batches]
    for ii in range(0, batch_size*n_batches, batch_size):
        data_batch = data[ii:ii + batch_size]
        target_batch = target[ii:ii + batch_size]
        
        yield data_batch, target_batch
        
        
def cal_accuracy(decoded, target):
    error = abs(decoded - target)
    Acc = (np.sum(error <= 0.08)/(decoded.shape[0]*decoded.shape[1]*decoded.shape[2]*decoded.shape[3]))*100
    
    return Acc

In [ ]:
num_layers = 2
epochs = 1800
batch_size = 80

for lr in [0.001]:
    save_string = './checkpoints_view_invariant/E2E_lr={}_bs={}.ckpt'.format(lr, batch_size)
    with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
        sess.run(tf.global_variables_initializer())

        iteration = 1
        mean_val_loss = 0
        mean_val_acc = 0
        count_loss_not_decrease_epochs = 0
        count_acc_not_decrease_epochs = 0
        Last_val_loss = 0
        Last_val_acc = 0


        for e in range(epochs):
            for train_batch, target_batch in get_batches(train_x, train_y, batch_size):

                start = time.time()

                feed_1 = {
                            model.inputs_1: train_batch, 
                            model.targets_1: target_batch,
                            model.keep_p: 0.6,
                            model.learning_rate: lr,
                            model.training:True
                        }

                train_loss, _, decoded_img = sess.run([model.cost, model.opt, model.decoded], feed_dict=feed_1)

                train_acc = cal_accuracy(decoded_img, target_batch)

                if iteration%25==0:
                    end = time.time()
                    print("Epoch: {}/{},".format(e+1, epochs),' '
                              "Iteration: {},".format(iteration),' '
                              "Train loss: {:.3f},".format(train_loss),'      '
                              "{:.1f}s /batch.".format((end-start)/5),' '
                              "Train Accuracy: %{:.3f}".format(train_acc))

                    ##############################################################
                    ######################## VALIDATION ##########################
                    ##############################################################

                if iteration%75==0:
                    validation_loss = []
                    validation_acc = []

                    if batch_size >= len(val_x):
                        val_batch_size = len(val_x)
                    else: 
                        val_batch_size = batch_size


                    for ii, (val_batch, val_target_batch) in enumerate(get_batches(val_x, val_y, val_batch_size)):
                        feed_2 = {
                                    model.inputs_1: val_batch,
                                    model.targets_1: val_target_batch,
                                    model.keep_p: 1,
                                    model.training:True
                                }

                        val_loss, val_decoded_img = sess.run([model.cost, model.decoded], 
                                                                              feed_dict=feed_2)

                        val_acc = cal_accuracy(val_decoded_img, val_target_batch)

                        validation_loss.append(val_loss)
                        validation_acc.append(val_acc)

                    Last_val_loss = mean_val_loss
                    Last_val_acc = mean_val_acc
                    mean_val_loss = np.mean(np.array(validation_loss))
                    mean_val_acc = np.mean(np.array(validation_acc))

                    print()
                    print("Validation loss: {:.3f},".format(mean_val_loss),' '
                              "Validation accuracy: {:.3f},".format(mean_val_acc))
    #                           "maxpool_3.shape: {}.".format(max_pool_3[0].shape))
                    print()
                    ####### plot #######
                if iteration%500==0:

                    feed_3 = {
                                model.inputs_1: val_x[:10],
                                model.keep_p: 1,
                                model.training:True
                                }

                    fig, axes = plt.subplots(nrows=2, ncols=10, sharex=True, sharey=True, figsize=(20,4))
                    in_imgs = val_y[:10]
                    reconstructed = sess.run(model.decoded, feed_dict=feed_3)

                        # plot out
                    for images, row in zip([in_imgs, reconstructed], axes):
                        for img, ax in zip(images, row):
                            ax.imshow(img.reshape((64, 64)), cmap='Greys_r')
                            ax.get_xaxis().set_visible(False)
                            ax.get_yaxis().set_visible(False)


                    fig.tight_layout(pad=0.1)

                iteration += 1

    #             # Early stopping  
    #             if mean_val_Acc - Last_val_Acc <= -0.3:
    #                 count_Acc_not_increase_epochs += 1
    #             if Last_val_loss - mean_val_loss <= -0.01:
    #                 count_loss_not_decrease_epochs += 1

    #             if mean_val_Acc - Last_val_Acc <= -2:
    #                 break
    #             if count_Acc_not_increase_epochs >= 10:
    #                 break
    #             if count_loss_not_decrease_epochs >= 10:
    #                 break

        model.saver.save(sess, r"{}".format(save_string))
        
print(' ')
print(' ')
print("leraning_rate={},num_layers={},batch_size={} finished, saved".format(lr, num_layers, batch_size))
print(' ')
print(' ')